In [3]:
import csv
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
import time
from bs4 import BeautifulSoup  # để loại bỏ các thẻ in đậm, nghiêng
import re
subject_files = {
    "Văn": "output/van.csv",
    "Sử": "output/lich_su.csv",
    "Địa": "output/dia_ly.csv",
    "Anh văn": "output/anhvan.csv"
}

In [4]:
# Mở trình duyệt
driver = webdriver.Chrome()

In [5]:
# Hàm làm sạch HTML
def clean_html_text(html):
    soup = BeautifulSoup(html, "html.parser")
    return soup.get_text(separator=" ", strip=True)

# Hàm lấy đáp án đúng và giải thích
def extract_answer_and_explanation(reason_html):
    soup = BeautifulSoup(reason_html, "html.parser")
    text = soup.get_text(separator="\n", strip=True)
    
    match = re.search(r"Đáp án(?: cần chọn là)?[:：]?\s*([A-D])", text, re.IGNORECASE)
    correct_answer = match.group(1).upper() if match else None
    
    if correct_answer:
        explanation_text = re.sub(r"Đáp án(?: cần chọn là)?[:：]?\s*"+re.escape(correct_answer), "", text, flags=re.IGNORECASE).strip()
    else:
        explanation_text = text.strip()
    
    cleaned_explanation = " ".join(explanation_text.split())
    explanation = cleaned_explanation if cleaned_explanation and not cleaned_explanation.startswith("Đáp án") else None
    
    return correct_answer, explanation

In [6]:
# Vòng lặp xử lý từng môn
for subject, filepath in subject_files.items():
    print(f"\nĐANG XỬ LÝ MÔN: {subject}")
    results = []
    # lấy sl câu hỏi
    count = 0

    with open(filepath, mode='r', encoding='utf-8-sig') as f:
        reader = csv.DictReader(f)
        for row in reader:
            if count >= 50:
                break
    
            title = row["Title"]
            link = row["Link"]
            
            print(f"\n🔗 Truy cập: {title}")
            driver.get(link)
            time.sleep(2)  # đợi trang load

            questions = driver.find_elements(By.CSS_SELECTOR, "div.quiz-answer-item")
            for q in questions:
                try:
                    question_html = q.find_element(By.CSS_SELECTOR, "div.title-question").get_attribute("innerHTML")
                    question_text = clean_html_text(question_html)
                    # question_html = q.find_element(By.CSS_SELECTOR, "div.title-question").get_attribute("innerHTML")
                    # question_text = BeautifulSoup(question_html, "html.parser").get_text(separator=" ", strip=True)
                    # Đáp án
                    answer_elems = q.find_elements(By.CSS_SELECTOR, "div.answers label")
                    answers = [ans.text.strip() for ans in answer_elems if ans.text.strip()]

                    # # Giải thích và đáp án đúng
                    # reason_elem = q.find_element(By.CLASS_NAME, "reason")
                    # reason_html = reason_elem.get_attribute("innerHTML")
                    # correct_answer, explanation = extract_answer_and_explanation(reason_html)
                    # --- LẤY GIẢI THÍCH ---
                    reason_elem = q.find_element(By.CLASS_NAME, "reason")
                    reason_html = reason_elem.get_attribute("innerHTML")
                    soup = BeautifulSoup(reason_html, "html.parser")
                    # Xóa toàn bộ phần "Xem thêm..." và các phần tử sau đó
                    unwanted = soup.find(string=re.compile(r"Xem thêm", re.IGNORECASE))
                    if unwanted:
                        # Loại bỏ phần tử cha chứa dòng "Xem thêm" và toàn bộ sau nó
                        parent = unwanted.find_parent()
                        if parent:
                            # Xóa tất cả các phần tử sau phần tử "Xem thêm"
                            for elem in parent.find_all_next():
                                elem.decompose()
                            parent.decompose()  # xóa luôn phần tử gốc

                    # Tiếp tục xử lý như cũ
                    full_text = soup.get_text(separator="\n", strip=True)

                    # --- TÌM ĐÁP ÁN ĐÚNG ---
                    match = re.search(r"Đáp án(?: đúng là)?[:：]?\s*([A-D])", full_text, re.IGNORECASE)
                    correct_answer = match.group(1).upper() if match else None

                    # --- TÁCH PHẦN GIẢI THÍCH ---
                    if correct_answer:
                        explanation_text = re.sub(r"Đáp án(?: đúng là)?[:：]?\s*" + re.escape(correct_answer), "", full_text, flags=re.IGNORECASE).strip()
                    else:
                        explanation_text = full_text.strip()

                    explanation_text = re.sub(r"\s{2,}", " ", explanation_text).strip()
                    explanation = explanation_text if explanation_text and not explanation_text.lower().startswith("đáp án") else None


                    result_item = {
                        "question": question_text,
                        "answers": answers,
                        "correct_answer": correct_answer
                    }

                    if explanation:
                        result_item["explanation"] = explanation

                    results.append(result_item)
                    count += 1
                except Exception as e:
                    continue

    # Lưu kết quả
    output_path = f"output/{subject.lower().replace(' ', '_')}.json"
    with open(output_path, "w", encoding="utf-8") as out_file:
        json.dump(results, out_file, ensure_ascii=False, indent=2)
    
    print(f"✅ Đã lưu {len(results)} câu hỏi vào: {output_path}")

# Đóng trình duyệt
driver.quit()


ĐANG XỬ LÝ MÔN: Văn

🔗 Truy cập: Trắc nghiệm Ngữ văn 9 Phong cách Hồ Chí Minh  (có đáp án)

🔗 Truy cập: Trắc nghiệm Tìm hiểu chung về tác phẩm Phong cách Hồ Chí Minh

🔗 Truy cập: Trắc nghiệm Phân tích chi tiết tác phẩm Phong cách Hồ Chí Minh

🔗 Truy cập: Trắc nghiệm Tổng hợp đề đọc hiểu văn bản Phong cách Hồ Chí Minh
✅ Đã lưu 61 câu hỏi vào: output/văn.json

ĐANG XỬ LÝ MÔN: Sử

🔗 Truy cập: Trắc nghiệm Lịch Sử 9 Bài 1 (có đáp án): Liên Xô và các nước Đông Âu từ 1945 đến giữa những năm 70 của thế kỉ XX

🔗 Truy cập: Trắc nghiệm Lịch Sử 9 Bài 2 (có đáp án): Liên Xô và các nước Đông Âu từ giữa những năm 70 đến đầu những năm 90 của thế kỉ XX

🔗 Truy cập: Trắc nghiệm Lịch Sử 9 Bài 3 (có đáp án): Quá trình phát triển của phong trào giải phóng dân tộc và sự tan rã của hệ thống thuộc địa
✅ Đã lưu 55 câu hỏi vào: output/sử.json

ĐANG XỬ LÝ MÔN: Địa

🔗 Truy cập: Trắc nghiệm Địa Lí 9 Bài 1 (có đáp án): Cộng đồng các dân tộc Việt Nam

🔗 Truy cập: Bài 1: Cộng đồng các dân tộc Việt Nam

🔗 Truy cập: T